In [3]:
# from pyhealth.datasets import MIMIC3Dataset, MIMIC4Dataset
# from GraphCare.task_fn import drug_recommendation_fn, drug_recommendation_mimic4_fn, mortality_prediction_mimic3_fn, readmission_prediction_mimic3_fn, length_of_stay_prediction_mimic3_fn, length_of_stay_prediction_mimic4_fn, mortality_prediction_mimic4_fn, readmission_prediction_mimic4_fn

# ds = MIMIC4Dataset(
# root="/data/physionet.org/files/mimiciv/2.0/hosp/", 
# tables=["diagnoses_icd", "procedures_icd", "prescriptions"],      
# code_mapping={
#     "NDC": ("ATC", {"target_kwargs": {"level": 3}}),
#     "ICD9CM": "CCSCM",
#     "ICD9PROC": "CCSPROC",
#     "ICD10CM": "CCSCM",
#     "ICD10PROC": "CCSPROC",
#     },
# dev=False
# )

# sample_dataset = ds.set_task(drug_recommendation_mimic4_fn)

In [15]:
task = "drugrec"

In [24]:
import pickle

with open(f'/data/pj20/exp_data/ccscm_ccsproc/sample_dataset_mimic4_{task}_th015_.pkl', 'rb') as f:
    sample_dataset = pickle.load(f)

In [25]:
train_ratios = \
[
    # 0.001,
    # 0.002,
    # 0.003,
    # 0.004,
    # 0.005,
    # 0.006,
    # 0.007,
    # 0.008,
    # 0.009,
    # 0.01,
    # 0.02,
    # 0.03,
    # 0.04,
    # 0.05,
    # 0.06,
    # 0.07,
    # 0.08,
    # 0.09,
    # 0.1,
    # 0.3,
    # 0.5,
    # 0.7,
    # 0.9,
    1.0,
]

In [26]:
from pyhealth.datasets import split_by_patient, get_dataloader

for train_ratio in train_ratios:
    train_dataset, val_dataset, test_dataset = split_by_patient(sample_dataset, [0.8, 0.1, 0.1], train_ratio=train_ratio, seed=528)
    train_loader = get_dataloader(train_dataset, batch_size=64, shuffle=True)
    val_loader = get_dataloader(val_dataset, batch_size=64, shuffle=False)
    test_loader = get_dataloader(test_dataset, batch_size=64, shuffle=False)


    from pyhealth.trainer import Trainer
    import torch
    from pyhealth.models import Transformer, RETAIN, SafeDrug, MICRON, CNN, RNN, GAMENet
    from collections import defaultdict

    results = defaultdict(list)

    for i in range(1):
        for model_ in [
            RNN,
            # Transformer, 
            # RETAIN,
            # SafeDrug,
            # MICRON,
            # GAMENet
            ]:
            try:
                model = model_(
                    dataset=sample_dataset,
                    feature_keys=["conditions", "procedures"],
                    label_key="drugs",
                    mode="multilabel",
                )
            except:
                model = model_(dataset=sample_dataset)

            device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

            ## binary
            # trainer = Trainer(model=model, device=device, metrics=["pr_auc", "roc_auc", "accuracy", "f1", "jaccard"])
            # trainer.train(
            #     train_dataloader=train_loader,
            #     val_dataloader=val_loader,
            #     epochs=5,
            #     monitor="accuracy",
            # )

            ## multi-label
            trainer = Trainer(model=model, device=device, metrics=["pr_auc_samples", "roc_auc_samples", "f1_samples", "jaccard_samples"])
            trainer.train(
                train_dataloader=train_loader,
                val_dataloader=val_loader,
                epochs=5,
                monitor="pr_auc_samples",
            )

            ## multi-class
            # trainer = Trainer(model=model, device=device, metrics=["roc_auc_weighted_ovr", "cohen_kappa", "accuracy", "f1_weighted"])
            # trainer.train(
            #     train_dataloader=train_loader,
            #     val_dataloader=val_loader,
            #     epochs=5,
            #     monitor="roc_auc_weighted_ovr",
            # )

            results[model_.__name__].append(trainer.evaluate(val_loader))

RNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(273, 128, padding_idx=0)
    (procedures): Embedding(216, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (rnn): ModuleDict(
    (conditions): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True)
    )
    (procedures): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True)
    )
  )
  (fc): Linear(in_features=256, out_features=191, bias=True)
)
Metrics: ['pr_auc_samples', 'roc_auc_samples', 'f1_samples', 'jaccard_samples']
Device: cuda:1

Training:
Batch size: 64
Optimizer: <class 'torch.optim.adam.Adam'>
Optimizer params: {'lr': 0.001}
Weight decay: 0.0
Max grad norm: None
Val dataloader: <torch.utils.data.dataloader.DataLoader object at 0x7fb588a3ce50>
Monitor: pr_auc_samples
Monitor criterion: max
Epochs: 5

Epoch 0 / 5: 100%|██████████| 120/120 [00:01<00:00, 74.45it/s]
--- Train epoch-0, step-120 ---
loss

In [27]:
avg_results = defaultdict(dict)

for k, v in results.items():
    for k_, v_ in v[0].items():
        avg_results[k][k_] = sum([vv[k_] for vv in v]) / len(v)

In [28]:
import numpy as np
# calculate standard deviation
variation_results = defaultdict(dict)

for k, v in results.items():
    for k_, v_ in v[0].items():
        variation_results[k][k_] = np.std([vv[k_] for vv in v])

In [29]:
avg_results

defaultdict(dict,
            {'RNN': {'pr_auc_samples': 0.6906467428253514,
              'roc_auc_samples': 0.9246048623270761,
              'f1_samples': 0.5444197656781944,
              'jaccard_samples': 0.3916203471958493,
              'loss': 0.22034245133399963}})

In [30]:
variation_results

defaultdict(dict,
            {'RNN': {'pr_auc_samples': 0.0,
              'roc_auc_samples': 0.0,
              'f1_samples': 0.0,
              'jaccard_samples': 0.0,
              'loss': 0.0}})